In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm
import warnings
import re
import csv
import os
from datetime import datetime
import glob
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from xml.etree.ElementTree import fromstring 
import shutil
import warnings
import random
import xml.etree.ElementTree as ET

In [2]:
# Remove old data and database

if os.path.exists("data"):
    shutil.rmtree("data")
if os.path.exists("ufc_database.db"):
    os.remove("ufc_database.db")
os.makedirs("data")

# # Create required directories
# directories = ['./data/fighters/', './data/git/']
# for directory in directories:
#     os.makedirs(directory, exist_ok=True)

# # Define CSV files and their columns
# csv_files = {
#     './data/event_urls_sherdog.csv': ['Event_URL'],
#     './data/fighter_id_sherdog.csv': ['Fighter', 'Fighter_ID'],
#     './data/event_data_sherdog.csv': ['Event Name', 'Event Location', 'Event Date', 
#                                       'Fighter 1', 'Fighter 2', 'Weight Class', 
#                                       'Winning Fighter', 'Winning Method', 
#                                       'Winning Round', 'Winning Time', 'Referee'],
#     './data/fighter_info.csv': ['Fighter', 'Nickname', 'Birth Date', 'Nationality', 
#                                 'Hometown', 'Association', 'Weight Class', 'Height', 
#                                 'REACH', 'STANCE', 'Wins', 'Losses', 'Win_Decision', 
#                                 'Win_KO', 'Win_Sub', 'Loss_Decision', 'Loss_KO', 
#                                 'Loss_Sub', 'Sherdog URL', 'BFO URL']
# }

# # Ensure CSV files exist and have headers
# for file_path, columns in csv_files.items():
#     if not os.path.exists(file_path):
#         pd.DataFrame(columns=columns).to_csv(file_path, index=False)

# # Function to download GitHub data
# def download_github_files():
#     files = ["ufc_event_details.csv", "ufc_fight_results.csv", 
#              "ufc_fight_stats.csv", "ufc_fighter_tott.csv"]
#     base_url = "https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/"
#     save_path = "./data/git/"

#     for file in files:
#         response = requests.get(base_url + file)
#         if response.status_code == 200:
#             with open(os.path.join(save_path, file), 'wb') as f:
#                 f.write(response.content)
#         else:
#             print(f"Failed to download {file}")

# # Execute file download
# download_github_files()


In [3]:
# Sherdog: Event URLs
# event_urls_sherdog.csv

urls = [
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/1',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/2',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/3',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/4',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/5',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/6',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/7'
]
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0"
]
headers = {"User-Agent": random.choice(user_agents)}
df = pd.read_csv('./data/event_urls_sherdog.csv') if os.path.isfile('./data/event_urls_sherdog.csv') else pd.DataFrame(columns=['Event_URL'])
existing_urls = set(df['Event_URL'])

print(f"Starting to scrape {len(urls)} URLs...")
for i, url in enumerate(urls, 1):
    print(f"Processing URL {i}/{len(urls)}")
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_div = soup.find('div', {'class': 'single_tab', 'id': 'recent_tab'})
        if specific_div:
            new_urls = [a.get('href') for a in specific_div.find_all('a', itemprop='url') if a.get('href') and a.get('href') not in existing_urls]
            df = pd.concat([df, pd.DataFrame(new_urls, columns=['Event_URL'])], ignore_index=True)
    except requests.RequestException:
        print(f"Failed to process URL {i}")
        pass

df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Updated event URLs saved.")

# Remove Broken URL's 
urls_to_delete = {
    "/events/UFC-233-Ultimate-Fighting-Championship-233-72021",
    "/events/UFC-Fight-Night-97-Lamas-vs-Penn-90890",
    "/events/UFC-176-Aldo-vs-Mendes-2-37609",
    "/events/UFC-151-Jones-vs-Henderson-25809"
}
df = pd.read_csv("./data/event_urls_sherdog.csv")
df = df[~df["Event_URL"].isin(urls_to_delete)]
df.to_csv("./data/event_urls_sherdog.csv", index=False)
print("Broken URLs removed.")

# Event URLs 
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
]
df = pd.read_csv('./data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
event_dates = []
with requests.Session() as session:
    session.headers.update({"User-Agent": random.choice(user_agents)})
    with ThreadPoolExecutor(max_workers=40) as executor:
        # futures = {executor.submit(session.get, url, {"User-Agent": random.choice(user_agents)}, 30): url for url in df['Event_URL']}
        futures = {executor.submit(lambda url: session.get(url, headers={"User-Agent": random.choice(user_agents)}, timeout=30), url): url for url in df['Event_URL']}
        for future in tqdm(as_completed(futures), total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
            url = futures[future]
            try:
                response = future.result()
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    event_date_meta = soup.find('meta', itemprop='startDate')
                    event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
                else:
                    event_dates.append("Error")
            except requests.exceptions.RequestException:
                event_dates.append("Error")
df['Event Date'] = event_dates
df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Event dates appended successfully.")
print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
print(f"Column names: {list(df.columns)}")


Starting to scrape 7 URLs...
Processing URL 1/7
Processing URL 2/7
Processing URL 3/7
Processing URL 4/7
Processing URL 5/7
Processing URL 6/7
Processing URL 7/7
Updated event URLs saved.
Broken URLs removed.


Scraping Events:   0%|          | 0/696 [00:00<?, ?Event/s]

Event dates appended successfully.
Total number of rows including the header in event_urls_sherdog.csv: 696
Column names: ['Event_URL', 'Event Date']


In [4]:
# Sherdog: Event Data
# event_data_sherdog.csv

warnings.filterwarnings("ignore", category=FutureWarning)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
urls_df = pd.read_csv('data/event_urls_sherdog.csv')
all_data = []
def fetch_event_data(url, session):
    full_url = f'https://sherdog.com{url}' if not url.startswith('http') else url
    event_data = []
    try:
        with session.get(full_url, headers=headers) as response:
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                event_name = soup.find('span', itemprop='name').text.strip()
                event_location = soup.find('span', itemprop='location').text.strip()
                event_date = soup.find('meta', itemprop='startDate')['content'].strip()
                main_event_fighters = soup.find_all('div', class_='fighter')
                if main_event_fighters:
                    fighter1 = main_event_fighters[0].find('span', itemprop='name').text.strip()
                    fighter2 = main_event_fighters[1].find('span', itemprop='name').text.strip()
                    fighter1_id = main_event_fighters[0].find('a', itemprop='url')['href'].split('-')[-1]
                    fighter2_id = main_event_fighters[1].find('a', itemprop='url')['href'].split('-')[-1]
                    weight_class = soup.find('span', class_='weight_class').text.strip()
                    winning_fighter = fighter1  
                    winning_method_em = soup.find('em', string='Method').parent
                    winning_method = winning_method_em.contents[2].strip()
                    winning_round_em = soup.find('em', string='Round').parent
                    winning_round = winning_round_em.contents[2].strip()
                    winning_time_em = soup.find('em', string='Time').parent
                    winning_time = winning_time_em.contents[2].strip()
                    referee_em = soup.find('em', string='Referee').parent
                    referee = referee_em.find('a').text.strip()
                    event_data.append({
                        'Event Name': event_name,
                        'Event Location': event_location,
                        'Event Date': event_date,
                        'Fighter 1': fighter1,
                        'Fighter 2': fighter2,
                        'Fighter 1 ID': fighter1_id,
                        'Fighter 2 ID': fighter2_id,
                        'Weight Class': weight_class,
                        'Winning Fighter': winning_fighter,
                        'Winning Method': winning_method,
                        'Winning Round': winning_round,
                        'Winning Time': winning_time,
                        'Referee': referee,
                        'Fight Type': 'Main Event'
                    })
                other_bouts = soup.find_all('tr', itemprop='subEvent')
                for bout in other_bouts:
                    fighters = bout.find_all('div', class_='fighter_list')
                    if len(fighters) >= 2:
                        fighter1 = fighters[0].find('img')['title']
                        fighter2 = fighters[1].find('img')['title']
                        fighter1_url = fighters[0].find('a', itemprop='url')['href']
                        fighter2_url = fighters[1].find('a', itemprop='url')['href']
                        fighter1_id = fighter1_url.split('-')[-1]
                        fighter2_id = fighter2_url.split('-')[-1]
                        weight_class = bout.find('span', class_='weight_class')
                        weight_class = weight_class.text.strip() if weight_class else "Unknown"
                        winning_method = bout.find('td', class_='winby').find('b').get_text(strip=True)
                        winning_round = bout.find_all('td')[-2].get_text(strip=True)
                        winning_time = bout.find_all('td')[-1].get_text(strip=True)
                        referee = bout.find('td', class_='winby').find('a').get_text(strip=True)
                        event_data.append({
                            'Event Name': event_name,
                            'Event Location': event_location,
                            'Event Date': event_date,
                            'Fighter 1': fighter1,
                            'Fighter 2': fighter2,
                            'Fighter 1 ID': fighter1_id,
                            'Fighter 2 ID': fighter2_id,
                            'Weight Class': weight_class,
                            'Winning Fighter': fighter1,  
                            'Winning Method': winning_method,
                            'Winning Round': winning_round,
                            'Winning Time': winning_time,
                            'Referee': referee,
                            'Fight Type': 'Undercard'
                        })
        return event_data
    except Exception as e:
        print(f"Request failed for {full_url}: {e}")
        return None
session = requests.Session()
total_urls = len(urls_df['Event_URL'])
completed_requests = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    futures = [executor.submit(fetch_event_data, url, session) for url in urls_df['Event_URL']]
    for future in concurrent.futures.as_completed(futures):
        data = future.result()
        completed_requests += 1
        progress_percentage = (completed_requests / total_urls) * 100
        print(f"Completed {completed_requests}/{total_urls} requests ({progress_percentage:.2f}%)")
        if data:
            all_data.extend(data)
df = pd.DataFrame(all_data)
file_path = './data/event_data_sherdog.csv'
write_mode = 'a' if os.path.isfile(file_path) else 'w'
df.to_csv(file_path, mode=write_mode, header=not os.path.isfile(file_path), index=False)
print("Data successfully written to data/event_data_sherdog.csv")
print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv(file_path, index=False)

Completed 1/696 requests (0.14%)
Completed 2/696 requests (0.29%)
Completed 3/696 requests (0.43%)
Completed 4/696 requests (0.57%)
Completed 5/696 requests (0.72%)
Completed 6/696 requests (0.86%)
Completed 7/696 requests (1.01%)
Completed 8/696 requests (1.15%)
Completed 9/696 requests (1.29%)
Completed 10/696 requests (1.44%)
Completed 11/696 requests (1.58%)
Completed 12/696 requests (1.72%)
Completed 13/696 requests (1.87%)
Completed 14/696 requests (2.01%)
Completed 15/696 requests (2.16%)
Completed 16/696 requests (2.30%)
Completed 17/696 requests (2.44%)
Completed 18/696 requests (2.59%)
Completed 19/696 requests (2.73%)
Completed 20/696 requests (2.87%)
Completed 21/696 requests (3.02%)
Completed 22/696 requests (3.16%)
Completed 23/696 requests (3.30%)
Completed 24/696 requests (3.45%)
Completed 25/696 requests (3.59%)
Completed 26/696 requests (3.74%)
Completed 27/696 requests (3.88%)
Completed 28/696 requests (4.02%)
Completed 29/696 requests (4.17%)
Completed 30/696 reques

In [5]:
# Sherdog: Fighter IDs
# fighter_id_sherdog.csv

df = pd.read_csv('./data/event_data_sherdog.csv')
df2 = pd.DataFrame(columns=['Fighter', 'Fighter_ID'])
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
for index, row in df.iterrows():
    fighter1 = row['Fighter 1']
    fighter2 = row['Fighter 2']
    fighter1_id = row['Fighter 1 ID']
    fighter2_id = row['Fighter 2 ID']
    for fighter, fighter_id in zip([fighter1, fighter2], [fighter1_id, fighter2_id]):
        if fighter not in df2['Fighter'].values and fighter_id not in df2['Fighter_ID'].values:
            df2 = pd.concat([df2, pd.DataFrame([{'Fighter': fighter, 'Fighter_ID': fighter_id}])])  # adjusted line
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
print("Data successfully written to data/fighter_id_sherdog.csv")
print(f"Total number of rows: {len(df2)}")
print(f"Column names: {list(df2.columns)}")

# Remove nicknames
df = pd.read_csv('./data/fighter_id_sherdog.csv')
df['Fighter'] = df['Fighter'].str.replace(r" '.+?'", "", regex=True)
df.to_csv('./data/fighter_id_sherdog.csv', index=False)
df = pd.read_csv('./data/event_data_sherdog.csv')
for col in ['Fighter 1', 'Fighter 2', 'Winning Fighter']:
    df[col] = df[col].str.replace(r" '.+?'", "", regex=True)
df.to_csv('./data/event_data_sherdog.csv', index=False)

# Add 'UFC' indicator to current fighters in fighter_id_sherdog.csv.csv
df = pd.read_csv('./data/fighter_id_sherdog.csv')
df['UFC'] = 'y'
df.to_csv('./data/fighter_id_sherdog.csv', index=False)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv('./data/fighter_id_sherdog.csv', index=False)


Data successfully written to data/fighter_id_sherdog.csv
Total number of rows: 2420
Column names: ['Fighter', 'Fighter_ID']


In [6]:
# Sherdog: Fighter Info
# fighter_info.csv

def scrape_fighter_general_info_sherdog(fighter, fighter_id):
    url = f'https://www.sherdog.com/fighter/{fighter_id}'
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {}
    soup = BeautifulSoup(response.content, 'html.parser')
    fighter_dict = {}
    try:
        fighter_data = soup.find('div', class_='fighter-data')
    except AttributeError:
        fighter_data = None
    try:
        birthdate = soup.find('span', itemprop='birthDate')
        birthdate = (birthdate.text).strip('""')
    except AttributeError:
        birthdate = '-'
    try:
        nationality = soup.find('strong', itemprop='nationality')
        nationality = (nationality.text).strip()
    except AttributeError:
        nationality = '-'
    try:
        hometown = soup.find('span', {'itemprop': 'addressLocality'}).text
        hometown = hometown.strip()
    except AttributeError:
        hometown = '-'
    try:
        association = soup.find('span', {'itemprop': 'name'}).text
        association = association.strip()
    except AttributeError:
        association = '-'
    try:
        weight_class_div = fighter_data.find('div', {'class': 'association-class'})
        links = weight_class_div.find_all('a')
        weight_class = links[-1].text
        weight_class = weight_class.strip()
    except (AttributeError, IndexError):
        weight_class = ''
    try:
        nickname = soup.find('span', class_='nickname')
        nickname = (nickname.text).strip('"')
    except AttributeError:
        nickname = '-'
    try:
        height = soup.find('b', itemprop='height')
        height = (height.text).strip('"')
    except AttributeError:
        height = '-'
    try:
        wins = soup.find('div', class_='winloses win').find_all('span')[1]
        wins = (wins.text).strip()
    except AttributeError:
        wins = '-'
    try:
        losses = soup.find('div', class_='winloses lose').find_all('span')[1]
        losses = (losses.text).strip()
    except AttributeError:
        losses = '-'
    dec_data_list = []
    try:
        win_type = fighter_data.find_all('div', class_='meter-title', string='DECISIONS')
        for method in win_type:
            if method.text.startswith('DECISIONS'):
                dec_data = method.find_next('div', class_='pl').text
                dec_data_list.append(dec_data)
        wins_dec = (dec_data_list[0]).strip()
        losses_dec = (dec_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_dec = '-'
        losses_dec = '-'
    ko_data_list = []
    try:
        win_type = soup.find_all('div', class_='meter-title')
        for method in win_type:
            if method.text.startswith('KO'):
                ko_data = method.find_next('div', class_='pl').text
                ko_data_list.append(ko_data)
        wins_ko = (ko_data_list[0]).strip()
        losses_ko = (ko_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_ko = '-'
        losses_ko = '-'
    sub_data_list = []
    try:
        win_type = fighter_data.find_all('div', class_='meter-title', string='SUBMISSIONS')
        for method in win_type:
            if method.text.startswith('SUBMISSIONS'):
                sub_data = method.find_next('div', class_='pl').text
                sub_data_list.append(sub_data)
        wins_sub = (sub_data_list[0]).strip()
        losses_sub = (sub_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_sub = '-'
        losses_sub = '-'
    fighter_dict = {
        'Fighter': fighter,
        'Nickname': nickname,
        'Birth Date': birthdate,
        'Nationality': nationality,
        'Hometown': hometown,
        'Association': association,
        'Weight Class': weight_class,
        'Height': height,
        'Wins': wins,
        'Losses': losses,
        'Win_Decision': wins_dec,
        'Win_KO': wins_ko,
        'Win_Sub': wins_sub,
        'Loss_Decision': losses_dec,
        'Loss_KO': losses_ko,
        'Loss_Sub': losses_sub,
        'Fighter_ID': fighter_id
    }
    return fighter_dict

def scrape_fighters_concurrently():
    warnings.filterwarnings("ignore", category=FutureWarning)
    df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
    fighter_data_list = []
    total_fighters = len(df_fighter_id)
    fighters_processed = 0
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_fighter = {executor.submit(scrape_fighter_general_info_sherdog, row['Fighter'], row['Fighter_ID']): row for index, row in df_fighter_id.iterrows()}
        for future in tqdm(as_completed(future_to_fighter), total=len(future_to_fighter)):
            fighter_data = future.result()
            if fighter_data:
                fighter_data_list.append(fighter_data)
            fighters_processed += 1
            print(f"Scraping Fighter Info: {fighters_processed}/{total_fighters} fighters processed")
    new_df = pd.DataFrame(fighter_data_list)
    new_df.to_csv('./data/fighter_info.csv', index=False)
scrape_fighters_concurrently()

df = pd.read_csv('./data/fighter_info.csv')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv('./data/fighter_info.csv', index=False)
print("Data successfully written to data/fighter_info.csv")
print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")


  0%|          | 0/2420 [00:00<?, ?it/s]

Scraping Fighter Info: 1/2420 fighters processed
Scraping Fighter Info: 2/2420 fighters processed
Scraping Fighter Info: 3/2420 fighters processed
Scraping Fighter Info: 4/2420 fighters processed
Scraping Fighter Info: 5/2420 fighters processed
Scraping Fighter Info: 6/2420 fighters processed
Scraping Fighter Info: 7/2420 fighters processed
Scraping Fighter Info: 8/2420 fighters processed
Scraping Fighter Info: 9/2420 fighters processed
Scraping Fighter Info: 10/2420 fighters processed
Scraping Fighter Info: 11/2420 fighters processed
Scraping Fighter Info: 12/2420 fighters processed
Scraping Fighter Info: 13/2420 fighters processed
Scraping Fighter Info: 14/2420 fighters processed
Scraping Fighter Info: 15/2420 fighters processed
Scraping Fighter Info: 16/2420 fighters processed
Scraping Fighter Info: 17/2420 fighters processed
Scraping Fighter Info: 18/2420 fighters processed
Scraping Fighter Info: 19/2420 fighters processed
Scraping Fighter Info: 20/2420 fighters processed
Scraping 

In [7]:
# Sherdog: Fighters
# data/fighters/*

os.makedirs('./data/fighters/', exist_ok=True)
def scrape_fighter_fights_sherdog(fighter_name, fighter_id, fighter_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(fighter_url, headers=headers, timeout=60)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'new_table fighter'})
        rows = table.find_all('tr')[1:]
        fight_data = []
        new_opponents = []
        for row in rows:
            cols = row.find_all('td')
            fight_dict = {
                'Result': cols[0].text.strip(),
                'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
                'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
                'Method/Referee': cols[3].text.strip().split('\n')[0],
                'Rounds': cols[4].text.strip(),
                'Time': cols[5].text.strip()
            }
            fight_data.append(fight_dict)
            opponent_link = cols[1].find('a')['href'] if cols[1].find('a') else None
            if opponent_link:
                opponent_id = opponent_link.split('-')[-1]
                new_opponents.append({'Fighter': fight_dict['Opponent'], 'Fighter_ID': opponent_id})
        # time.sleep(0.2)  # Sleep 0.2 seconds after scraping each fighter page
        return fighter_id, fighter_name, fight_data, new_opponents
    return fighter_id, fighter_name, [], []
df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
all_new_opponents = []
def process_fighter(row):
    fighter_url = f"https://www.sherdog.com/fighter/{row['Fighter'].replace(' ', '-')}-{row['Fighter_ID']}"
    return scrape_fighter_fights_sherdog(row['Fighter'], row['Fighter_ID'], fighter_url)
total_fighters = len(df_fighter_id)
fighters_processed = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_fighter, row) for _, row in df_fighter_id.iterrows()]
    for future in as_completed(futures):
        fighter_id, fighter_name, fight_data, new_opponents = future.result()
        fighters_processed += 1
        print(f"Processed {fighters_processed}/{total_fighters} fighters.")
        if fight_data:
            pd.DataFrame(fight_data).to_csv(f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv", index=False)
            all_new_opponents.extend(new_opponents)
if all_new_opponents:
    df_new_opponents = pd.DataFrame(all_new_opponents).drop_duplicates()
    df_fighter_id = pd.concat([df_fighter_id, df_new_opponents], ignore_index=True).drop_duplicates()
    df_fighter_id.to_csv('./data/fighter_id_sherdog.csv', index=False)

# # os.makedirs('./data/fighters/')
# # from concurrent.futures import ThreadPoolExecutor, as_completed
# # def scrape_fighter_fights_sherdog(fighter_name, fighter_id, fighter_url):
# #     headers = {
# #         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
# #     }
# #     response = requests.get(fighter_url, headers=headers, timeout=(10, 30))
# #     if response.status_code == 200:
# #         soup = BeautifulSoup(response.content, 'html.parser')
# #         table = soup.find('table', {'class': 'new_table fighter'})
# #         rows = table.find_all('tr')[1:]
# #         fight_data = []
# #         new_opponents = []
# #         for row in rows:
# #             cols = row.find_all('td')
# #             fight_dict = {
# #                 'Result': cols[0].text.strip(),
# #                 'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
# #                 'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
# #                 'Method/Referee': cols[3].text.strip().split('\n')[0],
# #                 'Rounds': cols[4].text.strip(),
# #                 'Time': cols[5].text.strip()
# #             }
# #             fight_data.append(fight_dict)
# #             opponent_link = cols[1].find('a')['href'] if cols[1].find('a') else None
# #             if opponent_link:
# #                 opponent_id = opponent_link.split('-')[-1]
# #                 new_opponents.append({'Fighter': fight_dict['Opponent'], 'Fighter_ID': opponent_id})
# #         return fighter_id, fighter_name, fight_data, new_opponents
# #     return fighter_id, fighter_name, [], []
# # df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
# # all_new_opponents = []
# # def process_fighter(row):
# #     fighter_url = f"https://www.sherdog.com/fighter/{row['Fighter'].replace(' ', '-')}-{row['Fighter_ID']}"
# #     return scrape_fighter_fights_sherdog(row['Fighter'], row['Fighter_ID'], fighter_url)
# # total_fighters = len(df_fighter_id)
# # fighters_processed = 0
# # with ThreadPoolExecutor(max_workers=10) as executor:
# #     futures = [executor.submit(process_fighter, row) for _, row in df_fighter_id.iterrows()]
# #     for future in as_completed(futures):
# #     # for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping Fighter Data", unit="fighter"):
# #         time.sleep(0.1)  # Adding a small sleep between request submissions
# #         fighter_id, fighter_name, fight_data, new_opponents = future.result()
# #         fighters_processed += 1
# #         print(f"Processed {fighters_processed}/{total_fighters} fighters: {fighter_name}")
# #         if fight_data:
# #             pd.DataFrame(fight_data).to_csv(f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv", index=False)
# #             all_new_opponents.extend(new_opponents)
# # if all_new_opponents:
# #     df_new_opponents = pd.DataFrame(all_new_opponents).drop_duplicates()
# #     df_fighter_id = pd.concat([df_fighter_id, df_new_opponents], ignore_index=True).drop_duplicates()
# #     df_fighter_id.to_csv('./data/fighter_id_sherdog.csv', index=False)

# # directory_path = './data/fighters/'
# # for filename in os.listdir(directory_path):
# #     if filename.endswith('.csv'):  # Check if the file is a CSV
# #         file_path = os.path.join(directory_path, filename)
# #         df = pd.read_csv(file_path)
# #         df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
# #         df.to_csv(file_path, index=False)

# # If you want a progress bar, uncomment these lines:
# # from tqdm.notebook import tqdm

# user_agents = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0"
# ]

# os.makedirs('./data/fighters/', exist_ok=True)

# def scrape_fighter_fights_sherdog(fighter_name, fighter_id, fighter_url, max_attempts=3):
#     # Retries up to max_attempts times if there's a RequestException
#     for attempt in range(max_attempts):
#         headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
#         }
#         try:
#             # Separate connect/read timeouts, e.g., 10s to connect, 30s to read
#             response = requests.get(fighter_url, headers=headers, timeout=(10, 30))
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.content, 'html.parser')
#                 table = soup.find('table', {'class': 'new_table fighter'})
#                 if not table:
#                     return fighter_id, fighter_name, [], []

#                 rows = table.find_all('tr')[1:]
#                 fight_data = []
#                 new_opponents = []

#                 for row in rows:
#                     cols = row.find_all('td')
#                     fight_dict = {
#                         'Result': cols[0].text.strip(),
#                         'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
#                         'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
#                         'Method/Referee': cols[3].text.strip().split('\n')[0],
#                         'Rounds': cols[4].text.strip(),
#                         'Time': cols[5].text.strip()
#                     }
#                     fight_data.append(fight_dict)

#                     opponent_link = cols[1].find('a')['href'] if cols[1].find('a') else None
#                     if opponent_link:
#                         opponent_id = opponent_link.split('-')[-1]
#                         new_opponents.append({'Fighter': fight_dict['Opponent'], 'Fighter_ID': opponent_id})

#                 return fighter_id, fighter_name, fight_data, new_opponents
#             # If it's not a 200, just break to handle below, or keep looping?
#             # time.sleep(3)  # small backoff before next attempt
#         except requests.RequestException:
#             # If there's a network error, wait and retry
#             if attempt < max_attempts - 1:
#                 time.sleep(5)
#     # If all attempts fail or status never 200, return empty
#     return fighter_id, fighter_name, [], []

# # Load known fighter IDs
# fighter_id_path = './data/fighter_id_sherdog.csv'
# df_fighter_id = pd.read_csv(fighter_id_path)

# all_new_opponents = []

# def process_fighter(row):
#     fighter_url = f"https://www.sherdog.com/fighter/{row['Fighter'].replace(' ', '-')}-{row['Fighter_ID']}"
#     return scrape_fighter_fights_sherdog(row['Fighter'], row['Fighter_ID'], fighter_url)

# total_fighters = len(df_fighter_id)
# fighters_processed = 0

# with ThreadPoolExecutor(max_workers=5) as executor:
#     futures = [executor.submit(process_fighter, row) for _, row in df_fighter_id.iterrows()]

#     # If you want a progress bar, uncomment the tqdm usage:
#     # for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping Fighter Data", unit="fighter"):
#     for future in as_completed(futures):
#         # Add a small sleep (100ms) between processed fighters to avoid flooding
#         time.sleep(0.5)
#         fighter_id, fighter_name, fight_data, new_opponents = future.result()
#         fighters_processed += 1
#         print(f"Processed {fighters_processed}/{total_fighters} fighters: {fighter_name}")

#         if fight_data:
#             file_name = f"{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
#             pd.DataFrame(fight_data).to_csv(f"./data/fighters/{file_name}", index=False)
#             all_new_opponents.extend(new_opponents)

# # Append new opponents to fighter_id_sherdog.csv
# if all_new_opponents:
#     df_new_opponents = pd.DataFrame(all_new_opponents).drop_duplicates()
#     df_fighter_id = pd.concat([df_fighter_id, df_new_opponents], ignore_index=True).drop_duplicates()
#     df_fighter_id.to_csv(fighter_id_path, index=False)

# # Clean up each CSV in data/fighters/
# directory_path = './data/fighters/'
# for filename in os.listdir(directory_path):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(directory_path, filename)
#         df = pd.read_csv(file_path)
#         df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#         df.to_csv(file_path, index=False)

# print("Finished scraping fighter fights.")


Processed 1/2420 fighters.
Processed 2/2420 fighters.
Processed 3/2420 fighters.
Processed 4/2420 fighters.
Processed 5/2420 fighters.
Processed 6/2420 fighters.
Processed 7/2420 fighters.
Processed 8/2420 fighters.
Processed 9/2420 fighters.
Processed 10/2420 fighters.
Processed 11/2420 fighters.
Processed 12/2420 fighters.
Processed 13/2420 fighters.
Processed 14/2420 fighters.
Processed 15/2420 fighters.
Processed 16/2420 fighters.
Processed 17/2420 fighters.
Processed 18/2420 fighters.
Processed 19/2420 fighters.
Processed 20/2420 fighters.
Processed 21/2420 fighters.
Processed 22/2420 fighters.
Processed 23/2420 fighters.
Processed 24/2420 fighters.
Processed 25/2420 fighters.
Processed 26/2420 fighters.
Processed 27/2420 fighters.
Processed 28/2420 fighters.
Processed 29/2420 fighters.
Processed 30/2420 fighters.
Processed 31/2420 fighters.
Processed 32/2420 fighters.
Processed 33/2420 fighters.
Processed 34/2420 fighters.
Processed 35/2420 fighters.
Processed 36/2420 fighters.
P

In [8]:
# Test

first_name = "Dustin"
last_name = "Poirier"
file_pattern = f'./data/fighters/*{first_name}*{last_name}*.csv'
matching_files = glob.glob(file_pattern)
if matching_files:
    first_file = matching_files[0]
    df = pd.read_csv(first_file)
    print(f"Total number of rows including the header in {first_file}: {len(df)}")
    print(f"Column names: {list(df.columns)}")
else:
    print("No files found matching the pattern.")


Total number of rows including the header in ./data/fighters/Dustin_Poirier_50529.csv: 40
Column names: ['Result', 'Opponent', 'Event Date', 'Method/Referee', 'Rounds', 'Time']


In [11]:
# Sherdog: Cleaning

fighter_info_df = pd.read_csv('./data/fighter_info.csv')
event_data_df = pd.read_csv('./data/event_data_sherdog.csv')
print(f"Number of rows in fighter_info.csv: {len(fighter_info_df)}")
print(f"Number of rows in event_data_sherdog.csv: {len(event_data_df)}")

# Removing women's fights and fighters
womens_weight_classes = ['Strawweight'] # 'Flyweight', 'Bantamweight', 'Featherweight'
fighter_info_df = pd.read_csv('./data/fighter_info.csv')
cleaned_fighter_info_df = fighter_info_df[~fighter_info_df['Weight Class'].isin(womens_weight_classes)]
cleaned_fighter_info_df.to_csv('./data/fighter_info_men_only.csv', index=False)
print(f"Cleaned dataset saved to {'./data/fighter_info_men_only.csv'}")
event_data_df = pd.read_csv('./data/event_data_sherdog.csv')
cleaned_event_data_df = event_data_df[~event_data_df['Weight Class'].isin(womens_weight_classes)]
cleaned_event_data_df.to_csv('./data/event_data_sherdog_men_only.csv', index=False)
print(f"Cleaned event dataset saved to {'./data/event_data_sherdog_men_only.csv'}")
print(f"Number of rows in fighter_info_men_only.csv: {len(cleaned_fighter_info_df)}")
print(f"Number of rows in event_data_sherdog_men_only.csv: {len(cleaned_event_data_df)}")

Number of rows in fighter_info.csv: 2420
Number of rows in event_data_sherdog.csv: 7767
Cleaned dataset saved to ./data/fighter_info_men_only.csv
Cleaned event dataset saved to ./data/event_data_sherdog_men_only.csv
Number of rows in fighter_info_men_only.csv: 2323
Number of rows in event_data_sherdog_men_only.csv: 7454


In [10]:
!open data/fighter_info.csv
!open data/event_data_sherdog.csv

---

In [45]:
# UFC: Fighters

# Sitemaps
if os.path.exists("data/ufc_roster.csv"): os.remove("data/ufc_roster.csv")
ns = {
    "s": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "xhtml": "http://www.w3.org/1999/xhtml"
}
all_fighters = []
page = 1
consecutive_empty_pages = 0
max_consecutive_empty = 100
while True:
    sitemap_url = f"https://www.ufc.com/sitemap.xml?page={page}"
    try:
        response = requests.get(sitemap_url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"Page {page} does not exist. Stopping.")
        break
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page}: {e}")
        page += 1
        continue
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as e:
        print(f"Error parsing page {page}: {e}")
        if "not well-formed (invalid token)" in str(e):
            print("Reached end of valid pages. Stopping.")
            break
        page += 1
        continue
    fighter_urls = []
    for url_elem in root.findall("s:url", ns):
        loc_elem = url_elem.find("s:loc", ns)
        if loc_elem is not None:
            url_text = loc_elem.text
            if "/athlete/" in url_text:
                fighter_urls.append(url_text)
    print(f"https://www.ufc.com/sitemap.xml?page={page} -> {len(fighter_urls)} fighter URLs")
    if len(fighter_urls) == 0:
        consecutive_empty_pages += 1
        if consecutive_empty_pages >= max_consecutive_empty:
            print(f"Found {max_consecutive_empty} consecutive empty pages. Stopping.")
            break
    else:
        consecutive_empty_pages = 0

    for url in fighter_urls:
        try:
            slug = url.split("/athlete/")[1]
            name = slug.replace("-", " ").title()
        except IndexError:
            name = ""
        all_fighters.append({"name": name, "url": url})
    page += 1
print(f"Total fighter records found: {len(all_fighters)}")
csv_filename = "data/ufc_roster.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["name", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for fighter in all_fighters:
        writer.writerow(fighter)
print(f"All fighter names and URLs saved to {csv_filename}")
df = pd.read_csv("data/ufc_roster.csv")
all_fighter_urls = df["url"].tolist()
print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")


# # Fighter Pages
# if os.path.exists("data/ufc_fighter_profiles.csv"): os.remove("data/ufc_fighter_profiles.csv")
# fighter_data = pd.DataFrame(columns=['name', 'url'])
# for idx, url in enumerate(all_fighter_urls, start=1):
#     print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
#     resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
#     if resp.status_code != 200:
#         print(f"Error fetching fighter page: {url}")
#         continue
#     soup = BeautifulSoup(resp.text, "html.parser")
#     name_tag = soup.find("h1")
#     name = name_tag.get_text(strip=True) if name_tag else "Unknown"
#     fighter_data.loc[len(fighter_data)] = [name, url]
#     if idx % 100 == 0:
#         fighter_data.to_csv("data/ufc_fighter_profiles.csv", index=False)
#         print(f"Saved {len(fighter_data)} fighter profiles to data/ufc_fighter_profiles.csv")
# if len(fighter_data) > 0:
#     fighter_data.to_csv("data/ufc_fighter_profiles.csv", index=False)
#     print(f"Saved final {len(fighter_data)} fighter profiles to data/ufc_fighter_profiles.csv")


https://www.ufc.com/sitemap.xml?page=1 -> 26 fighter URLs
https://www.ufc.com/sitemap.xml?page=2 -> 74 fighter URLs
https://www.ufc.com/sitemap.xml?page=3 -> 43 fighter URLs
https://www.ufc.com/sitemap.xml?page=4 -> 48 fighter URLs
https://www.ufc.com/sitemap.xml?page=5 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=6 -> 56 fighter URLs
https://www.ufc.com/sitemap.xml?page=7 -> 12 fighter URLs
https://www.ufc.com/sitemap.xml?page=8 -> 190 fighter URLs
https://www.ufc.com/sitemap.xml?page=9 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=10 -> 33 fighter URLs
https://www.ufc.com/sitemap.xml?page=11 -> 17 fighter URLs
https://www.ufc.com/sitemap.xml?page=12 -> 510 fighter URLs
https://www.ufc.com/sitemap.xml?page=13 -> 67 fighter URLs
https://www.ufc.com/sitemap.xml?page=14 -> 116 fighter URLs
https://www.ufc.com/sitemap.xml?page=15 -> 774 fighter URLs
https://www.ufc.com/sitemap.xml?page=16 -> 1000 fighter URLs
https://www.ufc.com/sitemap.xml?page=17 -> 587 fighter URLs

In [46]:
roster_csv = "data/ufc_roster.csv"
if not os.path.exists(roster_csv):
    print(f"Error: {roster_csv} not found. Run the roster scraper first.")
    exit()
df = pd.read_csv(roster_csv)
print(f"Columns in {roster_csv}: {df.columns.tolist()}")
if "profile_url" in df.columns:
    all_fighter_urls = df["profile_url"].dropna().tolist()
elif "url" in df.columns:  
    all_fighter_urls = df["url"].dropna().tolist()
else:
    print("Error: No 'profile_url' or 'url' column found in the roster CSV.")
    exit()
print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")
os.makedirs("data", exist_ok=True)
profile_csv = "data/ufc_fighter_profiles.csv"
if os.path.exists(profile_csv): 
    os.remove(profile_csv)
fieldnames = ["name", "nickname", "weight_class", "record", "height", "reach", "stance", "dob", "profile_url", "thumbnail"]
with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, url in enumerate(all_fighter_urls, start=1):
        print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code != 200:
            print(f"Error fetching fighter page: {url}")
            continue
        soup = BeautifulSoup(resp.text, "html.parser")
        name_tag = soup.find("h1", class_="hero-profile__name")
        name = name_tag.get_text(strip=True) if name_tag else "Unknown"
        nickname_tag = soup.find("p", class_="hero-profile__nickname")
        nickname = nickname_tag.get_text(strip=True) if nickname_tag else ""
        record_tag = soup.find("p", class_="hero-profile__record")
        record = record_tag.get_text(strip=True) if record_tag else ""
        weight_class_tag = soup.find("p", class_="hero-profile__division-title")
        weight_class = weight_class_tag.get_text(strip=True) if weight_class_tag else ""
        thumbnail_tag = soup.find("img", class_="hero-profile__image")
        thumbnail = thumbnail_tag["src"] if thumbnail_tag and "src" in thumbnail_tag.attrs else ""
        stats = {}
        for stat in soup.find_all("div", class_="c-bio__info"):
            label = stat.find("div", class_="c-bio__label")
            value = stat.find("div", class_="c-bio__text")
            if label and value:
                stats[label.get_text(strip=True).lower()] = value.get_text(strip=True)
        height = stats.get("height", "")
        reach = stats.get("reach", "")
        stance = stats.get("stance", "")
        dob = stats.get("date of birth", "")
        fighter_data = {
            "name": name,
            "nickname": nickname,
            "weight_class": weight_class,
            "record": record,
            "height": height,
            "reach": reach,
            "stance": stance,
            "dob": dob,
            "profile_url": url,
            "thumbnail": thumbnail
        }
        writer.writerow(fighter_data)
        if idx % 100 == 0:
            print(f"Saved {idx} fighter profiles to {profile_csv}")
print(f"All fighter profiles saved to {profile_csv}")


Columns in data/ufc_roster.csv: ['name', 'url']
Total fighter URLs loaded from CSV: 3900
Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
Scraping fighter 3/3900: https://www.ufc.com/athlete/joshua-culibao
Scraping fighter 4/3900: https://www.ufc.com/athlete/david-dvorak
Scraping fighter 5/3900: https://www.ufc.com/athlete/darrick-minner
Scraping fighter 6/3900: https://www.ufc.com/athlete/chi-lewis-parry
Scraping fighter 7/3900: https://www.ufc.com/athlete/zhalgas-zhumagulov
Scraping fighter 8/3900: https://www.ufc.com/athlete/steve-garcia
Scraping fighter 9/3900: https://www.ufc.com/athlete/joel-alvarez
Scraping fighter 10/3900: https://www.ufc.com/athlete/mariya-agapova
Scraping fighter 11/3900: https://www.ufc.com/athlete/umar-nurmagomedov
Scraping fighter 12/3900: https://www.ufc.com/athlete/oleksandr-doskalchuk
Scraping fighter 13/3900: https://www.ufc.com/athlete/dong-hyun-kim-0
Scraping fighter 

KeyboardInterrupt: 

In [43]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

# Load fighter URLs from the existing UFC roster CSV file
roster_csv = "data/ufc_roster.csv"
if not os.path.exists(roster_csv):
    print(f"Error: {roster_csv} not found. Run the roster scraper first.")
    exit()

# Debugging Step: Print CSV column names
df = pd.read_csv(roster_csv)
print(f"Columns in {roster_csv}: {df.columns.tolist()}")

# Check if 'profile_url' exists, otherwise use a fallback
if "profile_url" in df.columns:
    all_fighter_urls = df["profile_url"].dropna().tolist()
elif "url" in df.columns:  # Some versions may use "url" instead of "profile_url"
    all_fighter_urls = df["url"].dropna().tolist()
else:
    print("Error: No 'profile_url' or 'url' column found in the roster CSV.")
    exit()

print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")

# Ensure output directory exists
os.makedirs("data", exist_ok=True)

# Remove old fighter profile CSV if it exists
profile_csv = "data/ufc_fighter_profiles.csv"
if os.path.exists(profile_csv): 
    os.remove(profile_csv)

# Define fields for the CSV file
fieldnames = ["name", "nickname", "weight_class", "record", "height", "reach", "stance", "dob", "profile_url", "thumbnail"]

# Open CSV and start scraping
with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for idx, url in enumerate(all_fighter_urls, start=1):
        print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code != 200:
            print(f"Error fetching fighter page: {url}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")

        # Extract fighter's name
        name_tag = soup.find("h1", class_="hero-profile__name")
        name = name_tag.get_text(strip=True) if name_tag else "Unknown"

        # Extract fighter's nickname
        nickname_tag = soup.find("p", class_="hero-profile__nickname")
        nickname = nickname_tag.get_text(strip=True) if nickname_tag else ""

        # Extract fighter's record
        record_tag = soup.find("p", class_="hero-profile__record")
        record = record_tag.get_text(strip=True) if record_tag else ""

        # Extract weight class
        weight_class_tag = soup.find("p", class_="hero-profile__division-title")
        weight_class = weight_class_tag.get_text(strip=True) if weight_class_tag else ""

        # Extract profile picture
        thumbnail_tag = soup.find("img", class_="hero-profile__image")
        thumbnail = thumbnail_tag["src"] if thumbnail_tag and "src" in thumbnail_tag.attrs else ""

        # Extract stats from bio table
        stats = {}
        for stat in soup.find_all("div", class_="c-bio__info"):
            label = stat.find("div", class_="c-bio__label")
            value = stat.find("div", class_="c-bio__text")
            if label and value:
                stats[label.get_text(strip=True).lower()] = value.get_text(strip=True)

        height = stats.get("height", "")
        reach = stats.get("reach", "")
        stance = stats.get("stance", "")
        dob = stats.get("date of birth", "")

        # Save the fighter details
        fighter_data = {
            "name": name,
            "nickname": nickname,
            "weight_class": weight_class,
            "record": record,
            "height": height,
            "reach": reach,
            "stance": stance,
            "dob": dob,
            "profile_url": url,
            "thumbnail": thumbnail
        }
        print(fighter_data)

        writer.writerow(fighter_data)

        # Save progress every 100 fighters
        if idx % 100 == 0:
            print(f"Saved {idx} fighter profiles to {profile_csv}")

print(f"All fighter profiles saved to {profile_csv}")


Columns in data/ufc_roster.csv: ['name', 'url']
Total fighter URLs loaded from CSV: 3900
Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
{'name': 'Shanna Young', 'nickname': '"The Shanimal"', 'weight_class': 'Poids mouche féminins Division', 'record': '', 'height': '', 'reach': '', 'stance': '', 'dob': '', 'profile_url': 'https://www.ufc.com/athlete/shanna-young', 'thumbnail': 'https://dmxg5wxfqgb4u.cloudfront.net/styles/athlete_bio_full_body/s3/2022-08/YOUNG_SHANNA_L_08-20.png?itok=Gyh7-KnT'}
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
{'name': 'Spike Carlyle', 'nickname': '"The Alpha Ginger"', 'weight_class': 'Poids plume Division', 'record': '', 'height': '', 'reach': '', 'stance': '', 'dob': '', 'profile_url': 'https://www.ufc.com/athlete/spike-carlyle', 'thumbnail': 'https://dmxg5wxfqgb4u.cloudfront.net/styles/athlete_bio_full_body/s3/2020-11/71596%252Fprofile-galery%252Ffullbodyleft-picture%252FCARLYLE_SPIKE_L_02-29.png?itok=w-i1Xhvt'}
Scr

KeyboardInterrupt: 

In [36]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Get fighter URLs from ufc_roster.csv
df = pd.read_csv("data/ufc_roster.csv")
fighter_urls = df["url"].tolist()

fighters_data = []

for idx, url in enumerate(fighter_urls, start=1):
    print(f"Scraping fighter {idx}/{len(fighter_urls)}: {url}")
    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"Error fetching fighter page: {url}")
        continue

    soup = BeautifulSoup(resp.text, "html.parser")
    name_tag = soup.find("h1")
    name = name_tag.get_text(strip=True) if name_tag else "Unknown"
    
    fighters_data.append({
        "name": name,
        "url": url
    })
    
    # Save progress every 100 fighters
    if idx % 100 == 0:
        with open("data/ufc_fighter_profiles.csv", "w", newline="", encoding="utf-8") as csvfile:
            fieldnames = ["name", "url"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for fighter in fighters_data:
                writer.writerow(fighter)
        print(f"Saved progress: {len(fighters_data)} fighter profiles")

# Save final results
if fighters_data:
    with open("data/ufc_fighter_profiles.csv", "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["name", "url"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in fighters_data:
            writer.writerow(fighter)
    print(f"Saved {len(fighters_data)} fighter profiles to data/ufc_fighter_profiles.csv")

    


Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
Scraping fighter 3/3900: https://www.ufc.com/athlete/joshua-culibao
Scraping fighter 4/3900: https://www.ufc.com/athlete/david-dvorak
Scraping fighter 5/3900: https://www.ufc.com/athlete/darrick-minner
Scraping fighter 6/3900: https://www.ufc.com/athlete/chi-lewis-parry
Scraping fighter 7/3900: https://www.ufc.com/athlete/zhalgas-zhumagulov
Scraping fighter 8/3900: https://www.ufc.com/athlete/steve-garcia
Scraping fighter 9/3900: https://www.ufc.com/athlete/joel-alvarez
Scraping fighter 10/3900: https://www.ufc.com/athlete/mariya-agapova
Scraping fighter 11/3900: https://www.ufc.com/athlete/umar-nurmagomedov
Scraping fighter 12/3900: https://www.ufc.com/athlete/oleksandr-doskalchuk
Scraping fighter 13/3900: https://www.ufc.com/athlete/dong-hyun-kim-0
Scraping fighter 14/3900: https://www.ufc.com/athlete/manel-kape
Scraping fighter 15/3900: https://www.ufc

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_ufc_roster():
    # URL for the UFC roster page
    url = "https://www.ufc.com/athletes/all"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching page: Status code {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the main container for fighter listings
    roster_container = soup.find("ul", class_="l-flex--4col-1to4")
    if not roster_container:
        print("Could not find the roster container on the page.")
        return []
    
    fighter_items = roster_container.find_all("li", class_="l-flex__item")
    if not fighter_items:
        print("No fighter elements found on the page.")
        return []
    
    roster = []
    for item in fighter_items:
        # Extract fighter's name
        name_tag = item.find("span", class_="c-listing-athlete__name")
        name = name_tag.get_text(strip=True) if name_tag else "Unknown"
        
        # Extract fighter's nickname
        nickname_tag = item.find("span", class_="c-listing-athlete__nickname")
        nickname = ""
        if nickname_tag:
            # The nickname may be nested within a field container
            inner = nickname_tag.find("div", class_="field__item")
            if inner:
                nickname = inner.get_text(strip=True)
            else:
                nickname = nickname_tag.get_text(strip=True)
        
        # Extract fighter's weight class from c-listing-athlete__title element
        weight_class_tag = item.find("span", class_="c-listing-athlete__title")
        weight_class = ""
        if weight_class_tag:
            inner = weight_class_tag.find("div", class_="field__item")
            if inner:
                weight_class = inner.get_text(strip=True)
            else:
                weight_class = weight_class_tag.get_text(strip=True)
        
        # Extract fighter's record
        record_tag = item.find("span", class_="c-listing-athlete__record")
        record = record_tag.get_text(strip=True) if record_tag else ""
        
        # Extract fighter's profile URL from the flipcard action div
        action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
        profile_url = ""
        if action_div:
            a_tag = action_div.find("a", href=True)
            if a_tag:
                profile_url = a_tag["href"]
                if profile_url.startswith("/"):
                    profile_url = "https://www.ufc.com" + profile_url
        
        # Extract fighter's thumbnail image URL
        thumbnail_tag = item.find("div", class_="c-listing-athlete__thumbnail")
        thumbnail = ""
        if thumbnail_tag:
            img_tag = thumbnail_tag.find("img")
            if img_tag and img_tag.has_attr("src"):
                thumbnail = img_tag["src"]
        
        roster.append({
            "name": name,
            "nickname": nickname,
            "weight_class": weight_class,
            "record": record,
            "profile_url": profile_url,
            "thumbnail": thumbnail
        })
    
    return roster

def save_roster_to_csv(roster, filename="ufc_roster.csv"):
    fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in roster:
            writer.writerow(fighter)
    print(f"Roster saved to {filename}")

if __name__ == "__main__":
    fighters = scrape_ufc_roster()
    if fighters:
        # Optional: print some of the data for verification
        for fighter in fighters:
            print(f"Name: {fighter['name']}")
            print(f"Nickname: {fighter['nickname']}")
            print(f"Weight Class: {fighter['weight_class']}")
            print(f"Record: {fighter['record']}")
            print(f"Profile URL: {fighter['profile_url']}")
            print(f"Thumbnail: {fighter['thumbnail']}\n")
        save_roster_to_csv(fighters)
    else:
        print("No fighters were scraped.")


In [39]:
import os
import requests
from bs4 import BeautifulSoup
import csv

# Load the fighter URLs from the existing CSV file
with open("data/ufc_roster.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    all_fighter_urls = [row["url"] for row in reader]
print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")

# Remove old fighter profile CSV if it exists
profile_csv = "data/ufc_fighter_profiles.csv"
if os.path.exists(profile_csv):
    os.remove(profile_csv)

fighters_data = []

for idx, url in enumerate(all_fighter_urls, start=1):
    print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"Error fetching fighter page: {url}")
        continue

    soup = BeautifulSoup(resp.text, "html.parser")
    
    # Extract fighter's name
    name_tag = soup.find("h1")
    name = name_tag.get_text(strip=True) if name_tag else "Unknown"
    
    # Extract fighter's nickname
    nickname_tag = soup.find("p", class_="field--name-nickname")
    nickname = ""
    if nickname_tag:
        inner = nickname_tag.find("div", class_="field__item")
        if inner:
            nickname = inner.get_text(strip=True)
        else:
            nickname = nickname_tag.get_text(strip=True)
    
    # Extract fighter's weight class
    weight_class_tag = soup.find("div", class_="c-bio__info--weight")
    weight_class = ""
    if weight_class_tag:
        inner = weight_class_tag.find("div", class_="field__item")
        if inner:
            weight_class = inner.get_text(strip=True)
        else:
            weight_class = weight_class_tag.get_text(strip=True)
    
    # Extract fighter's record
    record_tag = soup.find("p", class_="field--name-record")
    record = record_tag.get_text(strip=True) if record_tag else ""
    
    # Extract fighter's thumbnail image URL
    thumbnail_tag = soup.find("div", class_="c-bio__image")
    thumbnail = ""
    if thumbnail_tag:
        img_tag = thumbnail_tag.find("img")
        if img_tag and img_tag.has_attr("src"):
            thumbnail = img_tag["src"]
    
    fighters_data.append({
        "name": name,
        "nickname": nickname,
        "weight_class": weight_class,
        "record": record,
        "profile_url": url,
        "thumbnail": thumbnail
    })
    
    # Save progress every 100 fighters
    if idx % 100 == 0:
        with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
            fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for fighter in fighters_data:
                writer.writerow(fighter)
        print(f"Saved progress: {len(fighters_data)} fighter profiles")

# Save final results
if fighters_data:
    with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in fighters_data:
            writer.writerow(fighter)
    print(f"Saved {len(fighters_data)} fighter profiles to {profile_csv}")


Total fighter URLs loaded from CSV: 3900
Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
Scraping fighter 3/3900: https://www.ufc.com/athlete/joshua-culibao
Scraping fighter 4/3900: https://www.ufc.com/athlete/david-dvorak
Scraping fighter 5/3900: https://www.ufc.com/athlete/darrick-minner
Scraping fighter 6/3900: https://www.ufc.com/athlete/chi-lewis-parry
Scraping fighter 7/3900: https://www.ufc.com/athlete/zhalgas-zhumagulov
Scraping fighter 8/3900: https://www.ufc.com/athlete/steve-garcia
Scraping fighter 9/3900: https://www.ufc.com/athlete/joel-alvarez
Scraping fighter 10/3900: https://www.ufc.com/athlete/mariya-agapova
Scraping fighter 11/3900: https://www.ufc.com/athlete/umar-nurmagomedov
Scraping fighter 12/3900: https://www.ufc.com/athlete/oleksandr-doskalchuk
Scraping fighter 13/3900: https://www.ufc.com/athlete/dong-hyun-kim-0
Scraping fighter 14/3900: https://www.ufc.com/athlete/manel-kape


KeyboardInterrupt: 

In [40]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# Hardcoded fighter URL for testing
test_fighter_url = "https://www.ufc.com/athlete/jon-jones"

# Scraping test fighter with retries
def fetch_page(url, retries=3, delay=5):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(delay)
    print("Failed to fetch the page after multiple attempts.")
    return None

print(f"Scraping fighter: {test_fighter_url}")
page_content = fetch_page(test_fighter_url)
if page_content:
    soup = BeautifulSoup(page_content, "html.parser")

    # Extract fighter's name
    name_tag = soup.find("h1")
    name = name_tag.get_text(strip=True) if name_tag else "Unknown"

    # Extract fighter's stats table
    stats = {}
    for stat in soup.find_all("div", class_="c-bio__info"):
        label = stat.find("div", class_="c-bio__label")
        value = stat.find("div", class_="c-bio__text")
        if label and value:
            stats[label.get_text(strip=True).lower()] = value.get_text(strip=True)

    # Assign values from extracted stats
    record = stats.get("record", "")
    weight_class = stats.get("weight class", "")
    height = stats.get("height", "")
    reach = stats.get("reach", "")
    stance = stats.get("stance", "")
    dob = stats.get("date of birth", "")

    # Print extracted data
    print("\nExtracted Fighter Data:")
    print(f"Name: {name}")
    print(f"URL: {test_fighter_url}")
    print(f"Record: {record}")
    print(f"Weight Class: {weight_class}")
    print(f"Height: {height}")
    print(f"Reach: {reach}")
    print(f"Stance: {stance}")
    print(f"DOB: {dob}")


Scraping fighter: https://www.ufc.com/athlete/jon-jones

Extracted Fighter Data:
Name: Jon Jones
URL: https://www.ufc.com/athlete/jon-jones
Record: 
Weight Class: 
Height: 
Reach: 
Stance: 
DOB: 


In [35]:
# UFC: Part 2: For each fighter URL, scrape the fighter’s page to extract details.

fighters_data = []
for idx, url in enumerate(all_fighter_urls, start=1):
    print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"Error fetching fighter page: {url}")
        continue

    soup = BeautifulSoup(resp.text, "html.parser")
    # Example: assume the fighter's name is in the first <h1> tag.
    name_tag = soup.find("h1")
    name = name_tag.get_text(strip=True) if name_tag else "Unknown"
    
    fighters_data.append({
        "name": name,
        "url": url
    })

# Part 3: Save all fighter records to a single CSV file.
csv_filename = "data/ufc_fighter_profiles.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["name", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for fighter in fighters_data:
        writer.writerow(fighter)

print(f"Saved {len(fighters_data)} fighter profiles to {csv_filename}")

Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
Scraping fighter 3/3900: https://www.ufc.com/athlete/joshua-culibao
Scraping fighter 4/3900: https://www.ufc.com/athlete/david-dvorak
Scraping fighter 5/3900: https://www.ufc.com/athlete/darrick-minner
Scraping fighter 6/3900: https://www.ufc.com/athlete/chi-lewis-parry
Scraping fighter 7/3900: https://www.ufc.com/athlete/zhalgas-zhumagulov
Scraping fighter 8/3900: https://www.ufc.com/athlete/steve-garcia
Scraping fighter 9/3900: https://www.ufc.com/athlete/joel-alvarez
Scraping fighter 10/3900: https://www.ufc.com/athlete/mariya-agapova
Scraping fighter 11/3900: https://www.ufc.com/athlete/umar-nurmagomedov
Scraping fighter 12/3900: https://www.ufc.com/athlete/oleksandr-doskalchuk
Scraping fighter 13/3900: https://www.ufc.com/athlete/dong-hyun-kim-0
Scraping fighter 14/3900: https://www.ufc.com/athlete/manel-kape
Scraping fighter 15/3900: https://www.ufc

KeyboardInterrupt: 

In [ ]:

import requests
from bs4 import BeautifulSoup
import csv

def scrape_ufc_roster():
    # URL for the UFC roster page
    url = "https://www.ufc.com/athletes/all"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
    # Send HTTP GET request to the UFC page
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching page: Status code {response.status_code}")
        return []
    
    # Parse the returned HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the main container for fighter listings (an unordered list)
    roster_container = soup.find("ul", class_="l-flex--4col-1to4")
    if not roster_container:
        print("Could not find the roster container on the page.")
        return []
    
    # Each fighter is listed as an <li> element
    fighter_items = roster_container.find_all("li", class_="l-flex__item")
    if not fighter_items:
        print("No fighter elements found on the page.")
        return []
    
    roster = []
    for item in fighter_items:
        # Extract the fighter's name from <span class="c-listing-athlete__name">
        name_tag = item.find("span", class_="c-listing-athlete__name")
        name = name_tag.get_text(strip=True) if name_tag else "Unknown"
        
        # Extract the fighter's profile URL from the first <a> tag within the action container
        action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
        profile_url = ""
        if action_div:
            a_tag = action_div.find("a", href=True)
            if a_tag:
                profile_url = a_tag["href"]
                if profile_url.startswith("/"):
                    profile_url = "https://www.ufc.com" + profile_url
        
        roster.append({"name": name, "url": profile_url})
    
    return roster

def save_roster_to_csv(roster, filename="ufc_roster.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["name", "url"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in roster:
            writer.writerow(fighter)
    print(f"Roster saved to {filename}")

# Run everything in one cell
if __name__ == "__main__":
    fighters = scrape_ufc_roster()
    if fighters:
        for fighter in fighters:
            print(f"Name: {fighter['name']}")
            print(f"Profile: {fighter['url']}\n")
        save_roster_to_csv(fighters)
    else:
        print("No fighters were scraped.")

In [44]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

# Load fighter URLs from the existing UFC roster CSV file
roster_csv = "data/ufc_roster.csv"
if not os.path.exists(roster_csv):
    print(f"Error: {roster_csv} not found. Run the roster scraper first.")
    exit()

# Debugging Step: Print CSV column names
df = pd.read_csv(roster_csv)
print(f"Columns in {roster_csv}: {df.columns.tolist()}")

# Check if 'profile_url' exists, otherwise use a fallback
if "profile_url" in df.columns:
    all_fighter_urls = df["profile_url"].dropna().tolist()
elif "url" in df.columns:  # Some versions may use "url" instead of "profile_url"
    all_fighter_urls = df["url"].dropna().tolist()
else:
    print("Error: No 'profile_url' or 'url' column found in the roster CSV.")
    exit()

print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")

# Ensure output directory exists
os.makedirs("data", exist_ok=True)

# Remove old fighter profile CSV if it exists
profile_csv = "data/ufc_fighter_profiles.csv"
if os.path.exists(profile_csv): 
    os.remove(profile_csv)

# Define fields for the CSV file
fieldnames = ["name", "nickname", "weight_class", "record", "height", "reach", "stance", "dob", "profile_url", "thumbnail"]

# Open CSV and start scraping
with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for idx, url in enumerate(all_fighter_urls, start=1):
        print(f"Scraping fighter {idx}/{len(all_fighter_urls)}: {url}")
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code != 200:
            print(f"Error fetching fighter page: {url}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")

        # Extract fighter's name
        name_tag = soup.find("h1", class_="hero-profile__name")
        name = name_tag.get_text(strip=True) if name_tag else "Unknown"

        # Extract fighter's nickname
        nickname_tag = soup.find("p", class_="hero-profile__nickname")
        nickname = nickname_tag.get_text(strip=True) if nickname_tag else ""

        # Extract fighter's record
        record_tag = soup.find("p", class_="hero-profile__record")
        record = record_tag.get_text(strip=True) if record_tag else ""

        # Extract weight class
        weight_class_tag = soup.find("p", class_="hero-profile__division-title")
        weight_class = weight_class_tag.get_text(strip=True) if weight_class_tag else ""

        # Extract profile picture
        thumbnail_tag = soup.find("img", class_="hero-profile__image")
        thumbnail = thumbnail_tag["src"] if thumbnail_tag and "src" in thumbnail_tag.attrs else ""

        # Extract stats from bio table
        stats = {}
        for stat in soup.find_all("div", class_="c-bio__info"):
            label = stat.find("div", class_="c-bio__label")
            value = stat.find("div", class_="c-bio__text")
            if label and value:
                stats[label.get_text(strip=True).lower()] = value.get_text(strip=True)

        height = stats.get("height", "")
        reach = stats.get("reach", "")
        stance = stats.get("stance", "")
        dob = stats.get("date of birth", "")

        # Save the fighter details
        fighter_data = {
            "name": name,
            "nickname": nickname,
            "weight_class": weight_class,
            "record": record,
            "height": height,
            "reach": reach,
            "stance": stance,
            "dob": dob,
            "profile_url": url,
            "thumbnail": thumbnail
        }

        writer.writerow(fighter_data)

        # Save progress every 100 fighters
        if idx % 100 == 0:
            print(f"Saved {idx} fighter profiles to {profile_csv}")

print(f"All fighter profiles saved to {profile_csv}")


Columns in data/ufc_roster.csv: ['name', 'url']
Total fighter URLs loaded from CSV: 3900
Scraping fighter 1/3900: https://www.ufc.com/athlete/shanna-young
Scraping fighter 2/3900: https://www.ufc.com/athlete/spike-carlyle
Scraping fighter 3/3900: https://www.ufc.com/athlete/joshua-culibao
Scraping fighter 4/3900: https://www.ufc.com/athlete/david-dvorak
Scraping fighter 5/3900: https://www.ufc.com/athlete/darrick-minner
Scraping fighter 6/3900: https://www.ufc.com/athlete/chi-lewis-parry
Scraping fighter 7/3900: https://www.ufc.com/athlete/zhalgas-zhumagulov
Scraping fighter 8/3900: https://www.ufc.com/athlete/steve-garcia
Scraping fighter 9/3900: https://www.ufc.com/athlete/joel-alvarez
Scraping fighter 10/3900: https://www.ufc.com/athlete/mariya-agapova
Scraping fighter 11/3900: https://www.ufc.com/athlete/umar-nurmagomedov
Scraping fighter 12/3900: https://www.ufc.com/athlete/oleksandr-doskalchuk
Scraping fighter 13/3900: https://www.ufc.com/athlete/dong-hyun-kim-0
Scraping fighter 

KeyboardInterrupt: 

In [ ]:
# ### WORKING PAGE ###

# import requests
# from bs4 import BeautifulSoup
# import csv

# def scrape_ufc_roster():
#     # URL for the UFC roster page
#     url = "https://www.ufc.com/athletes/all"
#     headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
#     response = requests.get(url, headers=headers)
#     if response.status_code != 200:
#         print(f"Error fetching page: Status code {response.status_code}")
#         return []
    
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     # Locate the main container for fighter listings
#     roster_container = soup.find("ul", class_="l-flex--4col-1to4")
#     if not roster_container:
#         print("Could not find the roster container on the page.")
#         return []
    
#     fighter_items = roster_container.find_all("li", class_="l-flex__item")
#     if not fighter_items:
#         print("No fighter elements found on the page.")
#         return []
    
#     roster = []
#     for item in fighter_items:
#         # Extract fighter's name
#         name_tag = item.find("span", class_="c-listing-athlete__name")
#         name = name_tag.get_text(strip=True) if name_tag else "Unknown"
        
#         # Extract fighter's nickname
#         nickname_tag = item.find("span", class_="c-listing-athlete__nickname")
#         nickname = ""
#         if nickname_tag:
#             # The nickname may be nested within a field container
#             inner = nickname_tag.find("div", class_="field__item")
#             if inner:
#                 nickname = inner.get_text(strip=True)
#             else:
#                 nickname = nickname_tag.get_text(strip=True)
        
#         # Extract fighter's weight class from c-listing-athlete__title element
#         weight_class_tag = item.find("span", class_="c-listing-athlete__title")
#         weight_class = ""
#         if weight_class_tag:
#             inner = weight_class_tag.find("div", class_="field__item")
#             if inner:
#                 weight_class = inner.get_text(strip=True)
#             else:
#                 weight_class = weight_class_tag.get_text(strip=True)
        
#         # Extract fighter's record
#         record_tag = item.find("span", class_="c-listing-athlete__record")
#         record = record_tag.get_text(strip=True) if record_tag else ""
        
#         # Extract fighter's profile URL from the flipcard action div
#         action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
#         profile_url = ""
#         if action_div:
#             a_tag = action_div.find("a", href=True)
#             if a_tag:
#                 profile_url = a_tag["href"]
#                 if profile_url.startswith("/"):
#                     profile_url = "https://www.ufc.com" + profile_url
        
#         # Extract fighter's thumbnail image URL
#         thumbnail_tag = item.find("div", class_="c-listing-athlete__thumbnail")
#         thumbnail = ""
#         if thumbnail_tag:
#             img_tag = thumbnail_tag.find("img")
#             if img_tag and img_tag.has_attr("src"):
#                 thumbnail = img_tag["src"]
        
#         roster.append({
#             "name": name,
#             "nickname": nickname,
#             "weight_class": weight_class,
#             "record": record,
#             "profile_url": profile_url,
#             "thumbnail": thumbnail
#         })
    
#     return roster

# def save_roster_to_csv(roster, filename="ufc_roster.csv"):
#     fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
#     with open(filename, "w", newline="", encoding="utf-8") as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()
#         for fighter in roster:
#             writer.writerow(fighter)
#     print(f"Roster saved to {filename}")

# if __name__ == "__main__":
#     fighters = scrape_ufc_roster()
#     if fighters:
#         # Optional: print some of the data for verification
#         for fighter in fighters:
#             print(f"Name: {fighter['name']}")
#             print(f"Nickname: {fighter['nickname']}")
#             print(f"Weight Class: {fighter['weight_class']}")
#             print(f"Record: {fighter['record']}")
#             print(f"Profile URL: {fighter['profile_url']}")
#             print(f"Thumbnail: {fighter['thumbnail']}\n")
#         save_roster_to_csv(fighters)
#     else:
#         print("No fighters were scraped.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_ufc_roster():
    roster = []
    # Read fighter URLs from the CSV file
    try:
        with open("ufc_roster.csv", newline="", encoding="utf-8") as csvfile:
            reader = csv.DictReader(csvfile)
            fighter_list = list(reader)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return []

    if not fighter_list:
        print("No fighter URLs found in the CSV file.")
        return []

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    for fighter in fighter_list:
        profile_url = fighter.get("url", "")
        if not profile_url:
            print(f"No URL found for fighter: {fighter.get('name', 'Unknown')}")
            continue

        response = requests.get(profile_url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching page for {fighter.get('name', 'Unknown')}: Status code {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract fighter's name
        name_tag = soup.find("span", class_="c-listing-athlete__name")
        name = name_tag.get_text(strip=True) if name_tag else fighter.get("name", "Unknown")

        # Extract fighter's nickname
        nickname_tag = soup.find("span", class_="c-listing-athlete__nickname")
        nickname = ""
        if nickname_tag:
            inner = nickname_tag.find("div", class_="field__item")
            if inner:
                nickname = inner.get_text(strip=True)
            else:
                nickname = nickname_tag.get_text(strip=True)

        # Extract fighter's weight class from c-listing-athlete__title element
        weight_class_tag = soup.find("span", class_="c-listing-athlete__title")
        weight_class = ""
        if weight_class_tag:
            inner = weight_class_tag.find("div", class_="field__item")
            if inner:
                weight_class = inner.get_text(strip=True)
            else:
                weight_class = weight_class_tag.get_text(strip=True)

        # Extract fighter's record
        record_tag = soup.find("span", class_="c-listing-athlete__record")
        record = record_tag.get_text(strip=True) if record_tag else ""

        # Use the fighter's URL from CSV as the profile URL
        final_profile_url = profile_url

        # Extract fighter's thumbnail image URL
        thumbnail_tag = soup.find("div", class_="c-listing-athlete__thumbnail")
        thumbnail = ""
        if thumbnail_tag:
            img_tag = thumbnail_tag.find("img")
            if img_tag and img_tag.has_attr("src"):
                thumbnail = img_tag["src"]

        roster.append({
            "name": name,
            "nickname": nickname,
            "weight_class": weight_class,
            "record": record,
            "profile_url": final_profile_url,
            "thumbnail": thumbnail
        })

    return roster

def save_roster_to_csv(roster, filename="ufc_roster.csv"):
    fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in roster:
            writer.writerow(fighter)
    print(f"Roster saved to {filename}")

if __name__ == "__main__":
    fighters = scrape_ufc_roster()
    if fighters:
        # Optional: print some of the data for verification
        for fighter in fighters:
            print(f"Name: {fighter['name']}")
            print(f"Nickname: {fighter['nickname']}")
            print(f"Weight Class: {fighter['weight_class']}")
            print(f"Record: {fighter['record']}")
            print(f"Profile URL: {fighter['profile_url']}")
            print(f"Thumbnail: {fighter['thumbnail']}\n")
        save_roster_to_csv(fighters)
    else:
        print("No fighters were scraped.")


Name: Danny Abbadi
Nickname: "The Assassin"
Weight Class: Lightweight
Record: 2-2-0 (W-L-D)
Profile URL: https://www.ufc.com/athlete/danny-abbadi
Thumbnail: /themes/custom/ufc/assets/img/no-profile-image.png

Name: Nariman Abbassov
Nickname: 
Weight Class: Lightweight
Record: 0-1-0 (W-L-D)
Profile URL: https://www.ufc.com/athlete/nariman-abbassov
Thumbnail: https://dmxg5wxfqgb4u.cloudfront.net/styles/teaser/s3/2022-09/ABBASOV_NARIMAN.png?h=a30e7339&itok=u--Wh_ZO

Name: Tank Abbott
Nickname: "Tank"
Weight Class: Heavyweight
Record: 8-10-0 (W-L-D)
Profile URL: https://www.ufc.com/athlete/tank-abbott
Thumbnail: /themes/custom/ufc/assets/img/no-profile-image.png

Name: Hamdy Abdelwahab
Nickname: "The Hammer"
Weight Class: Heavyweight
Record: 6-0-0 (W-L-D)
Profile URL: https://www.ufc.com/athlete/hamdy-abdelwahab
Thumbnail: https://dmxg5wxfqgb4u.cloudfront.net/styles/teaser/s3/2025-01/ABDELWAHAB_HAMDY_02-01.png?h=a30e7339&itok=CRu257sy

Name: Mansur Abdul-Malik
Nickname: 
Weight Class: Midd

In [17]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_all_ufc_roster():
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    all_fighters = []
    
    # Gender index: 1 for men, 2 for women
    for gender_index in range(1, 3):
        page = 0
        while True:
            url = f'https://www.ufc.com/athletes/all?filters%5B0%5D=status%3A23&gender={gender_index}&page={page}'
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Error fetching page: {url}, status: {response.status_code}")
                break
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the main container for fighter listings
            roster_container = soup.find("ul", class_="l-flex--4col-1to4")
            if not roster_container:
                print(f"No roster container found on page: {url}")
                break
            
            # Find all fighter items on the page
            fighter_items = roster_container.find_all("li", class_="l-flex__item")
            if not fighter_items:
                print(f"No fighter items found on page: {url}")
                break
            
            for item in fighter_items:
                # Extract fighter's name
                name_tag = item.find("span", class_="c-listing-athlete__name")
                name = name_tag.get_text(strip=True) if name_tag else "Unknown"
                
                # Extract fighter's nickname
                nickname_tag = item.find("span", class_="c-listing-athlete__nickname")
                nickname = ""
                if nickname_tag:
                    inner = nickname_tag.find("div", class_="field__item")
                    nickname = inner.get_text(strip=True) if inner else nickname_tag.get_text(strip=True)
                
                # Extract fighter's weight class
                weight_class_tag = item.find("span", class_="c-listing-athlete__title")
                weight_class = ""
                if weight_class_tag:
                    inner = weight_class_tag.find("div", class_="field__item")
                    weight_class = inner.get_text(strip=True) if inner else weight_class_tag.get_text(strip=True)
                
                # Extract fighter's record
                record_tag = item.find("span", class_="c-listing-athlete__record")
                record = record_tag.get_text(strip=True) if record_tag else ""
                
                # Extract fighter's profile URL
                action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
                profile_url = ""
                if action_div:
                    a_tag = action_div.find("a", href=True)
                    if a_tag:
                        profile_url = a_tag["href"]
                        if profile_url.startswith("/"):
                            profile_url = "https://www.ufc.com" + profile_url
                
                # Extract fighter's thumbnail image URL
                thumbnail_tag = item.find("div", class_="c-listing-athlete__thumbnail")
                thumbnail = ""
                if thumbnail_tag:
                    img_tag = thumbnail_tag.find("img")
                    if img_tag and img_tag.has_attr("src"):
                        thumbnail = img_tag["src"]
                
                fighter = {
                    "gender": "men" if gender_index == 1 else "women",
                    "name": name,
                    "nickname": nickname,
                    "weight_class": weight_class,
                    "record": record,
                    "profile_url": profile_url,
                    "thumbnail": thumbnail
                }
                all_fighters.append(fighter)
            
            print(f"Finished page {page} for gender {'men' if gender_index == 1 else 'women'}")
            page += 1
            
    return all_fighters

def save_roster_to_csv(roster, filename="ufc_roster_all.csv"):
    fieldnames = ["gender", "name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in roster:
            writer.writerow(fighter)
    print(f"Roster saved to {filename}")

if __name__ == "__main__":
    roster = scrape_all_ufc_roster()
    print(f"Scraped {len(roster)} fighters in total.")
    save_roster_to_csv(roster)


Finished page 0 for gender men
Finished page 1 for gender men
Finished page 2 for gender men
Finished page 3 for gender men
Finished page 4 for gender men
Finished page 5 for gender men
Finished page 6 for gender men
Finished page 7 for gender men
Finished page 8 for gender men
Finished page 9 for gender men
Finished page 10 for gender men
Finished page 11 for gender men
Finished page 12 for gender men
Finished page 13 for gender men
Finished page 14 for gender men
Finished page 15 for gender men
Finished page 16 for gender men
Finished page 17 for gender men
Finished page 18 for gender men
Finished page 19 for gender men
Finished page 20 for gender men
Finished page 21 for gender men
Finished page 22 for gender men
Finished page 23 for gender men
Finished page 24 for gender men
Finished page 25 for gender men
Finished page 26 for gender men
Finished page 27 for gender men
Finished page 28 for gender men
Finished page 29 for gender men
Finished page 30 for gender men
Finished page 31 f

In [18]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_all_ufc_roster():
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    all_fighters = []
    
    # Loop for both genders: gender=1 for men, gender=2 for women.
    for gender_index in range(1, 3):
        page = 0
        while True:
            url = f'https://www.ufc.com/athletes/all?filters%5B0%5D=status%3A23&gender={gender_index}&page={page}'
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Error fetching page {page} for gender {gender_index}: status {response.status_code}")
                break
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # The fighters are listed inside an unordered list with class l-flex--4col-1to4
            roster_container = soup.find("ul", class_="l-flex--4col-1to4")
            if not roster_container:
                print(f"No roster container found on page {page} for gender {gender_index}.")
                break
            
            fighter_items = roster_container.find_all("li", class_="l-flex__item")
            if not fighter_items:
                print(f"No fighter items found on page {page} for gender {gender_index}.")
                break
            
            for item in fighter_items:
                # Extract fighter's name
                name_tag = item.find("span", class_="c-listing-athlete__name")
                name = name_tag.get_text(strip=True) if name_tag else ""
                
                # Extract fighter's nickname
                nickname_tag = item.find("span", class_="c-listing-athlete__nickname")
                nickname = ""
                if nickname_tag:
                    inner = nickname_tag.find("div", class_="field__item")
                    nickname = inner.get_text(strip=True) if inner else nickname_tag.get_text(strip=True)
                
                # Extract fighter's weight class (raw text from the title element)
                weight_class_tag = item.find("span", class_="c-listing-athlete__title")
                weight_class = ""
                if weight_class_tag:
                    inner = weight_class_tag.find("div", class_="field__item")
                    weight_class = inner.get_text(strip=True) if inner else weight_class_tag.get_text(strip=True)
                
                # Extract fighter's record
                record_tag = item.find("span", class_="c-listing-athlete__record")
                record = record_tag.get_text(strip=True) if record_tag else ""
                
                # Extract fighter's profile URL from the flipcard action div
                action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
                profile_url = ""
                if action_div:
                    a_tag = action_div.find("a", href=True)
                    if a_tag:
                        profile_url = a_tag["href"]
                        if profile_url.startswith("/"):
                            profile_url = "https://www.ufc.com" + profile_url
                
                # Extract fighter's thumbnail image URL
                thumbnail_tag = item.find("div", class_="c-listing-athlete__thumbnail")
                thumbnail = ""
                if thumbnail_tag:
                    img_tag = thumbnail_tag.find("img")
                    if img_tag and img_tag.has_attr("src"):
                        thumbnail = img_tag["src"]
                
                fighter = {
                    "name": name,
                    "nickname": nickname,
                    "weight_class": weight_class,
                    "record": record,
                    "profile_url": profile_url,
                    "thumbnail": thumbnail
                }
                all_fighters.append(fighter)
            
            print(f"Finished page {page} for gender {gender_index}")
            page += 1

    return all_fighters

def save_roster_to_csv(roster, filename="ufc_roster.csv"):
    fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for fighter in roster:
            writer.writerow(fighter)
    print(f"Roster saved to {filename}")

if __name__ == "__main__":
    roster = scrape_all_ufc_roster()
    print(f"Scraped {len(roster)} fighters in total.")
    save_roster_to_csv(roster)


# import requests
# from bs4 import BeautifulSoup
# import csv

# headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
# all_fighters = []

# # Loop over genders: gender=1 for men, gender=2 for women.
# for gender_index in range(1, 3):
#     page = 0
#     while True:
#         url = f'https://www.ufc.com/athletes/all?filters%5B0%5D=status%3A23&gender={gender_index}&page={page}'
#         response = requests.get(url, headers=headers)
#         if response.status_code != 200:
#             print(f"Error fetching page {page} for gender {gender_index}: status {response.status_code}")
#             break
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Locate the fighter listings container
#         roster_container = soup.find("ul", class_="l-flex--4col-1to4")
#         if not roster_container:
#             print(f"No roster container found on page {page} for gender {gender_index}.")
#             break
        
#         fighter_items = roster_container.find_all("li", class_="l-flex__item")
#         if not fighter_items:
#             print(f"No fighter items found on page {page} for gender {gender_index}.")
#             break
        
#         for item in fighter_items:
#             # Extract fighter's name
#             name_tag = item.find("span", class_="c-listing-athlete__name")
#             name = name_tag.get_text(strip=True) if name_tag else ""
            
#             # Extract fighter's nickname
#             nickname_tag = item.find("span", class_="c-listing-athlete__nickname")
#             nickname = ""
#             if nickname_tag:
#                 inner = nickname_tag.find("div", class_="field__item")
#                 nickname = inner.get_text(strip=True) if inner else nickname_tag.get_text(strip=True)
            
#             # Extract fighter's weight class
#             weight_class_tag = item.find("span", class_="c-listing-athlete__title")
#             weight_class = ""
#             if weight_class_tag:
#                 inner = weight_class_tag.find("div", class_="field__item")
#                 weight_class = inner.get_text(strip=True) if inner else weight_class_tag.get_text(strip=True)
            
#             # Extract fighter's record
#             record_tag = item.find("span", class_="c-listing-athlete__record")
#             record = record_tag.get_text(strip=True) if record_tag else ""
            
#             # Extract fighter's profile URL
#             action_div = item.find("div", class_="c-listing-athlete-flipcard__action")
#             profile_url = ""
#             if action_div:
#                 a_tag = action_div.find("a", href=True)
#                 if a_tag:
#                     profile_url = a_tag["href"]
#                     if profile_url.startswith("/"):
#                         profile_url = "https://www.ufc.com" + profile_url
            
#             # Extract fighter's thumbnail image URL
#             thumbnail_tag = item.find("div", class_="c-listing-athlete__thumbnail")
#             thumbnail = ""
#             if thumbnail_tag:
#                 img_tag = thumbnail_tag.find("img")
#                 if img_tag and img_tag.has_attr("src"):
#                     thumbnail = img_tag["src"]
            
#             # Append the fighter's raw data to our list
#             fighter = {
#                 "name": name,
#                 "nickname": nickname,
#                 "weight_class": weight_class,
#                 "record": record,
#                 "profile_url": profile_url,
#                 "thumbnail": thumbnail
#             }
#             all_fighters.append(fighter)
        
#         print(f"Finished page {page} for gender {gender_index}")
#         page += 1

# print(f"Scraped {len(all_fighters)} fighters in total.")

# # Save the results to CSV
# filename = "ufc_roster_all.csv"
# fieldnames = ["name", "nickname", "weight_class", "record", "profile_url", "thumbnail"]
# with open(filename, "w", newline="", encoding="utf-8") as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for fighter in all_fighters:
#         writer.writerow(fighter)
# print(f"Roster saved to {filename}")


Finished page 0 for gender 1
Finished page 1 for gender 1
Finished page 2 for gender 1
Finished page 3 for gender 1
Finished page 4 for gender 1
Finished page 5 for gender 1
Finished page 6 for gender 1
Finished page 7 for gender 1
Finished page 8 for gender 1
Finished page 9 for gender 1
Finished page 10 for gender 1
Finished page 11 for gender 1
Finished page 12 for gender 1
Finished page 13 for gender 1
Finished page 14 for gender 1
Finished page 15 for gender 1
Finished page 16 for gender 1
Finished page 17 for gender 1
Finished page 18 for gender 1
Finished page 19 for gender 1
Finished page 20 for gender 1
Finished page 21 for gender 1
Finished page 22 for gender 1
Finished page 23 for gender 1
Finished page 24 for gender 1
Finished page 25 for gender 1
Finished page 26 for gender 1
Finished page 27 for gender 1
Finished page 28 for gender 1
Finished page 29 for gender 1
Finished page 30 for gender 1
Finished page 31 for gender 1
Finished page 32 for gender 1
Finished page 33 for

In [19]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import csv

def get_fighter_urls_from_sitemap(url):
    """
    Fetch a sitemap or sitemap index and extract all <loc> that contain '/athlete/'.
    """
    resp = requests.get(url)
    resp.raise_for_status()
    
    root = ET.fromstring(resp.content)
    ns = {"s": "http://www.sitemaps.org/schemas/sitemap/0.9"}
    fighter_urls = []
    
    # Some sitemaps are "index" type: <sitemap><loc>...</loc></sitemap>
    # Others are direct <url><loc>...</loc></url> with actual pages.
    # We'll try to handle both:
    
    # 1) If it’s a sitemap index (sub-sitemaps):
    #    <sitemap><loc>...</loc></sitemap>
    for sitemap_elem in root.findall("s:sitemap", ns):
        loc_tag = sitemap_elem.find("s:loc", ns)
        if loc_tag is not None:
            sub_url = loc_tag.text
            # Possibly parse the sub-sitemap
            # But you might see that some of these sub-sitemaps are directly fighter pages
            # or just more nested sitemaps. So you can recursively parse them:
            fighter_urls.extend(get_fighter_urls_from_sitemap(sub_url))
    
    # 2) If it’s a direct URL listing:
    #    <url><loc>...</loc></url>
    for url_elem in root.findall("s:url", ns):
        loc_tag = url_elem.find("s:loc", ns)
        if loc_tag is not None:
            loc = loc_tag.text
            if "/athlete/" in loc:
                fighter_urls.append(loc)
    
    return fighter_urls

def scrape_fighter_page(url):
    # Basic example of scraping the fighter's page
    resp = requests.get(url)
    if resp.status_code != 200:
        return None
    soup = BeautifulSoup(resp.text, "html.parser")
    
    name_tag = soup.find("h1")
    name = name_tag.get_text(strip=True) if name_tag else ""
    
    return {
        "url": url,
        "name": name
    }

def main():
    all_fighter_urls = []
    
    # We suspect the main sitemap is paginated up to, say, 20 or so.
    # We'll just loop until we get an error or no results:
    for page_number in range(1, 30):
        sitemap_url = f"https://www.ufc.com/sitemap.xml?page={page_number}"
        print(f"Fetching {sitemap_url}")
        resp = requests.get(sitemap_url)
        if resp.status_code != 200:
            print("No more pages or error. Stopping.")
            break
        
        # Parse the current page’s sitemap for fighter URLs
        root = ET.fromstring(resp.content)
        ns = {"s": "http://www.sitemaps.org/schemas/sitemap/0.9"}
        
        # We look for <url> entries that are direct fighter pages:
        current_fighter_urls = []
        for url_elem in root.findall("s:url", ns):
            loc_tag = url_elem.find("s:loc", ns)
            if loc_tag is not None:
                loc = loc_tag.text
                # If it's a sub-sitemap, parse it recursively:
                # (But if you see from your screenshot that these are direct fighter pages,
                # then just do the check for '/athlete/')
                if "/athlete/" in loc:
                    current_fighter_urls.append(loc)
        
        if not current_fighter_urls:
            print(f"No fighter URLs found on page {page_number}, possibly done.")
            # Could break or keep going to see if next pages have more
            # For safety, let's continue in case there's a gap
            continue
        
        all_fighter


In [20]:
import requests
import xml.etree.ElementTree as ET

# URL of one of the sitemap pages (page 16 as an example)
sitemap_url = "https://www.ufc.com/sitemap.xml?page=16"

# Send a GET request
response = requests.get(sitemap_url, headers={"User-Agent": "Mozilla/5.0"})
if response.status_code == 200:
    # Parse the XML content
    root = ET.fromstring(response.content)
    # Define namespaces as used in the sitemap file
    ns = {
        "s": "http://www.sitemaps.org/schemas/sitemap/0.9",
        "xhtml": "http://www.w3.org/1999/xhtml"
    }
    
    fighter_urls = []
    # Iterate over each <url> element in the sitemap
    for url_elem in root.findall("s:url", ns):
        loc_elem = url_elem.find("s:loc", ns)
        if loc_elem is not None:
            url_text = loc_elem.text
            # Filter for fighter pages that include '/athlete/' in the URL
            if "/athlete/" in url_text:
                fighter_urls.append(url_text)
    
    print(f"Found {len(fighter_urls)} fighter URLs on this sitemap page:")
    for url in fighter_urls:
        print(url)
else:
    print(f"Error fetching sitemap: status code {response.status_code}")


Found 1000 fighter URLs on this sitemap page:
https://www.ufc.com/athlete/bobby-southworth
https://www.ufc.com/athlete/jason-thacker
https://www.ufc.com/athlete/lodune-sincaid
https://www.ufc.com/athlete/alex-schoenauer
https://www.ufc.com/athlete/josh-raferty
https://www.ufc.com/athlete/heath-herring
https://www.ufc.com/athlete/rampage-jackson
https://www.ufc.com/athlete/mirko-cro-cop
https://www.ufc.com/athlete/chad-reiner
https://www.ufc.com/athlete/lyoto-machida
https://www.ufc.com/athlete/john-halverson
https://www.ufc.com/athlete/frankie-edgar
https://www.ufc.com/athlete/diego-saraiva
https://www.ufc.com/athlete/rex-holman
https://www.ufc.com/athlete/jason-gilliam
https://www.ufc.com/athlete/fabricio-werdum
https://www.ufc.com/athlete/wander-braga
https://www.ufc.com/athlete/roan-carneiro
https://www.ufc.com/athlete/thiago-tavares
https://www.ufc.com/athlete/naoyuki-kotani
https://www.ufc.com/athlete/justin-mccully
https://www.ufc.com/athlete/jimmy-dexter
https://www.ufc.com/athl

In [21]:
import requests
import xml.etree.ElementTree as ET
import csv

# Step 1: Download and parse the main sitemap index
def get_sub_sitemaps(main_sitemap_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(main_sitemap_url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching main sitemap: {response.status_code}")
        return []
    root = ET.fromstring(response.content)
    ns = {"s": "http://www.sitemaps.org/schemas/sitemap/0.9"}
    sub_sitemap_urls = []
    for sitemap in root.findall("s:sitemap", ns):
        loc_elem = sitemap.find("s:loc", ns)
        if loc_elem is not None:
            sub_sitemap_urls.append(loc_elem.text)
    return sub_sitemap_urls

# Step 2: For each sub-sitemap, download and extract fighter URLs
def get_fighter_urls_from_sitemap(sitemap_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(sitemap_url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching {sitemap_url}: {response.status_code}")
        return []
    root = ET.fromstring(response.content)
    ns = {"s": "http://www.sitemaps.org/schemas/sitemap/0.9"}
    fighter_urls = []
    for url_elem in root.findall("s:url", ns):
        loc_elem = url_elem.find("s:loc", ns)
        if loc_elem is not None and "/athlete/" in loc_elem.text:
            fighter_urls.append(loc_elem.text)
    return fighter_urls

def main():
    main_sitemap_url = "https://www.ufc.com/sitemap.xml"
    sub_sitemaps = get_sub_sitemaps(main_sitemap_url)
    print(f"Found {len(sub_sitemaps)} sub-sitemaps in the main sitemap.")
    
    all_fighter_urls = []
    for sitemap_url in sub_sitemaps:
        fighter_urls = get_fighter_urls_from_sitemap(sitemap_url)
        print(f"{sitemap_url} -> {len(fighter_urls)} fighter URLs")
        all_fighter_urls.extend(fighter_urls)
    
    # Remove duplicates if any
    all_fighter_urls = list(set(all_fighter_urls))
    print(f"Total fighter URLs found: {len(all_fighter_urls)}")
    
    # Save the fighter URLs to a CSV file
    with open("ufc_fighter_urls.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["fighter_url"])
        for url in all_fighter_urls:
            writer.writerow([url])
    print("Fighter URLs saved to ufc_fighter_urls.csv")

if __name__ == "__main__":
    main()


Found 55 sub-sitemaps in the main sitemap.
https://www.ufc.com/sitemap.xml?page=1 -> 26 fighter URLs
https://www.ufc.com/sitemap.xml?page=2 -> 74 fighter URLs
https://www.ufc.com/sitemap.xml?page=3 -> 43 fighter URLs
https://www.ufc.com/sitemap.xml?page=4 -> 48 fighter URLs
https://www.ufc.com/sitemap.xml?page=5 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=6 -> 56 fighter URLs
https://www.ufc.com/sitemap.xml?page=7 -> 12 fighter URLs
https://www.ufc.com/sitemap.xml?page=8 -> 190 fighter URLs
https://www.ufc.com/sitemap.xml?page=9 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=10 -> 33 fighter URLs
https://www.ufc.com/sitemap.xml?page=11 -> 17 fighter URLs
https://www.ufc.com/sitemap.xml?page=12 -> 510 fighter URLs
https://www.ufc.com/sitemap.xml?page=13 -> 67 fighter URLs
https://www.ufc.com/sitemap.xml?page=14 -> 116 fighter URLs
https://www.ufc.com/sitemap.xml?page=15 -> 774 fighter URLs
https://www.ufc.com/sitemap.xml?page=16 -> 1000 fighter URLs
https://www.ufc.

In [22]:
import requests
import xml.etree.ElementTree as ET
import csv
TOTAL_PAGES = 55
ns = {
    "s": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "xhtml": "http://www.w3.org/1999/xhtml"
}
all_fighters = []
for page in range(1, TOTAL_PAGES + 1):
    sitemap_url = f"https://www.ufc.com/sitemap.xml?page={page}"
    response = requests.get(sitemap_url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        print(f"Error fetching page {page}: status code {response.status_code}")
        continue
    root = ET.fromstring(response.content)
    fighter_urls = []
    for url_elem in root.findall("s:url", ns):
        loc_elem = url_elem.find("s:loc", ns)
        if loc_elem is not None:
            url_text = loc_elem.text
            if "/athlete/" in url_text:
                fighter_urls.append(url_text)
    print(f"Page {page}: Found {len(fighter_urls)} fighter URLs.")
    for url in fighter_urls:
        try:
            slug = url.split("/athlete/")[1]
            name = slug.replace("-", " ").title()
        except IndexError:
            name = ""
        all_fighters.append({"name": name, "url": url})
print(f"Total fighter records found: {len(all_fighters)}")
csv_filename = "data/ufc_roster.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["name", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for fighter in all_fighters:
        writer.writerow(fighter)
print(f"All fighter names and URLs saved to {csv_filename}")


Page 1: Found 26 fighter URLs.
Page 2: Found 74 fighter URLs.
Page 3: Found 43 fighter URLs.
Page 4: Found 48 fighter URLs.
Page 5: Found 60 fighter URLs.
Page 6: Found 56 fighter URLs.
Page 7: Found 12 fighter URLs.
Page 8: Found 190 fighter URLs.
Page 9: Found 60 fighter URLs.
Page 10: Found 33 fighter URLs.
Page 11: Found 17 fighter URLs.
Page 12: Found 510 fighter URLs.
Page 13: Found 67 fighter URLs.
Page 14: Found 116 fighter URLs.
Page 15: Found 774 fighter URLs.
Page 16: Found 1000 fighter URLs.
Page 17: Found 587 fighter URLs.
Page 18: Found 0 fighter URLs.
Page 19: Found 0 fighter URLs.
Page 20: Found 0 fighter URLs.
Page 21: Found 0 fighter URLs.
Page 22: Found 0 fighter URLs.
Page 23: Found 0 fighter URLs.
Page 24: Found 0 fighter URLs.
Page 25: Found 0 fighter URLs.
Page 26: Found 0 fighter URLs.
Page 27: Found 0 fighter URLs.
Page 28: Found 0 fighter URLs.
Page 29: Found 0 fighter URLs.
Page 30: Found 0 fighter URLs.
Page 31: Found 0 fighter URLs.
Page 32: Found 0 fighter

---

In [ ]:
# GitHub Data
# https://github.com/Greco1899/scrape_ufc_stats

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import csv
import warnings
import re
import sqlite3

os.makedirs('./data/github/', exist_ok=True)
os.makedirs('./data/github/fighter-details', exist_ok=True)

urls = [
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_event_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_results.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_stats.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fighter_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fighter_tott.csv'
]

for url in urls:
    df = pd.read_csv(url)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df.to_csv('./data/github/' + url.split('/')[-1], index=False)
    
# Merge the DATE and LOCATION of every fight

event_details_df = pd.read_csv('./data/github/ufc_event_details.csv')
fight_results_df = pd.read_csv('./data/github/ufc_fight_results.csv')

# Set the 'EVENT' column as index in the event details DataFrame for easier merging
event_details_df_for_merge = event_details_df.set_index('EVENT')

# Merge 'DATE' and 'LOCATION' from event details into fight results DataFrame based on 'EVENT'
merged_df = fight_results_df.join(event_details_df_for_merge[['DATE', 'LOCATION']], on='EVENT')

merged_df.to_csv('./data/github/master.csv', index=False)

# Create FIGHTER1, FIGHTER2, and WINNING_FIGHTER columns

fight_results_df = pd.read_csv('./data/github/master.csv')

# Split the 'BOUT' column into two new columns 'Fighter1' and 'Fighter2'
fight_results_df[['FIGHTER1', 'FIGHTER2']] = fight_results_df['BOUT'].str.split(' vs. ', expand=True)

# Use a multiline lambda within apply for readability without an external function
fight_results_df['WINNING_FIGHTER'] = fight_results_df.apply(
    lambda row: row['FIGHTER1'].strip() if row['OUTCOME'] == 'W/L' else
                (row['FIGHTER2'].strip() if row['OUTCOME'] == 'L/W' else
                 ('No Contest' if row['OUTCOME'] == 'NC/NC' else
                  ('Draw' if row['OUTCOME'] == 'D/D' else 'Unknown Outcome'))),
    axis=1
)

fight_results_df.to_csv('./data/github/master.csv', index=False)

# Load the latest master.csv file into a DataFrame
latest_master_df = pd.read_csv('./data/github/master.csv')

# Clean leading and trailing whitespace in FIGHTER1 and FIGHTER2 columns
latest_master_df['FIGHTER1'] = latest_master_df['FIGHTER1'].str.strip()
latest_master_df['FIGHTER2'] = latest_master_df['FIGHTER2'].str.strip()

# Create an SQLite database connection in a writable location
conn = sqlite3.connect('ufc_database.db')
cursor = conn.cursor()

# Create the fight_results table
create_table_query = """
CREATE TABLE IF NOT EXISTS fight_results (
    EVENT TEXT,
    BOUT TEXT,
    OUTCOME TEXT,
    WEIGHTCLASS TEXT,
    METHOD TEXT,
    ROUND INTEGER,
    TIME TEXT,
    TIME_FORMAT TEXT,
    REFEREE TEXT,
    DETAILS TEXT,
    URL TEXT,
    DATE TEXT,
    LOCATION TEXT,
    FIGHTER1 TEXT,
    FIGHTER2 TEXT,
    WINNING_FIGHTER TEXT
)
"""
cursor.execute(create_table_query)
conn.commit()

# Insert data into the fight_results table
latest_master_df.to_sql('fight_results', conn, if_exists='replace', index=False)

# Verify the data insertion
result = cursor.execute("SELECT COUNT(*) FROM fight_results").fetchone()[0]

# Close the connection
conn.close()

result
